In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, assemble
import numpy as np
import math
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from threading import Thread, Lock, Condition

In [2]:
num_players = 2
coin_tosses = pow(num_players, 2)
k = 3*np.log2(num_players)
num_qubits = math.ceil(k)
upper_bound = pow(num_players,3)

coin_lock = Lock()
leader_lock = Lock()

c_messages = []
l_messages = []

c_measurements = []
l_measurements = []

In [3]:
def coin_toss(player_id, condition):  

    coin = QuantumCircuit(num_players)

    coin.h(0)

    for qubit in range(1,num_players):
        coin.cx(0,qubit)

    coin.barrier()

    coin_lock.acquire()
    try:
        for receiver in range(num_players):
            coin_results = [player_id, receiver, coin]
            
            c_messages.append(coin_results)
            print(f'Player {player_id} tossed coin for player {receiver}')
    finally:
        coin_lock.release()   

    with condition:
        if(len(c_messages) == coin_tosses):
            condition.notify_all()

    return coin

In [4]:
def measure_coins(player_id):
    backend = Aer.get_backend('aer_simulator')

    coin_lock.acquire()
    
    for i in range(len(c_messages)):
        if player_id == c_messages[i][1]:

            c_messages[i][2].measure_all()

            print(c_messages[i][2])

            aer_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(c_messages[i][2], shots=1, memory=True)
            result = aer_sim.run(qobj).result()

            memory = result.get_memory()[0].replace(" ", "")
            # memory = result.get_memory()[0][-num_players:]
            measured_bit = int(memory)

            c_values = [player_id, c_messages[i][0], measured_bit]
            c_measurements.append(c_values)

            print(f"Player {player_id} measured {measured_bit} in player {c_messages[i][0]} circuit")
            
    coin_lock.release()

    return c_measurements
               
    

In [5]:
def leader_toss(player_id, condition):
    
    l_registers = []

    for a in range(num_players):
        q_name = 'q' + str(a)
        q_reg = QuantumRegister(num_qubits, q_name)
        l_registers.append(q_reg)

        # name = 'c' + str(a)
        # reg = ClassicalRegister(1, name)
        # l_registers.append(reg)

    leader = QuantumCircuit(*l_registers)

    for qubit in range(num_qubits):
        leader.h(l_registers[0][qubit])
    for a in range(1,num_players):
        for qubit in range(num_qubits):
            leader.cx(l_registers[0][qubit], l_registers[a][qubit])

    leader.barrier()   

    leader_lock.acquire()
    try:
        for receiver in range(num_players):
            leader_results = [player_id, receiver, leader]

            l_messages.append(leader_results)
            print(f'Player {player_id} chose leader for player {receiver}')
    finally:
        leader_lock.release()

    with condition:
        if(len(l_messages) == coin_tosses):
            condition.notify_all()   

    return leader
    

In [6]:
def measure_leader(player_id):
    backend = Aer.get_backend('aer_simulator')

    leader_lock.acquire()
    
    for i in range(len(l_messages)):
        if player_id == l_messages[i][1]:

            l_messages[i][2].measure_all()

            print(l_messages[i][2])

            aer_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(l_messages[i][2], shots=1, memory=True)
            result = aer_sim.run(qobj).result()

            memory = result.get_memory()[0].replace(" ", "")
            # memory = result.get_memory()[0][-num_players:]
            measured_bit = int(memory)
            l_value = [player_id, l_messages[i][0], measured_bit]
            l_measurements.append(l_value)

            print(f"Player {player_id} measured {measured_bit} in player {l_messages[i][0]} circuit")
            
    leader_lock.release()
    
    return l_measurements

    

In [7]:
# measured_str = max(l_hist)
# print(measured_str)
# measured_int = int(measured_str,2)
# print("Register output = %i" % measured_int)

In [8]:
def fail_stop_protocol(player_id, c_condition, l_condition):
    
    coin = coin_toss(player_id, c_condition)
    with c_condition:
        while(len(c_messages) != coin_tosses):
            c_condition.wait()

    leader = leader_toss(player_id, l_condition)
    with l_condition:
        while(len(l_messages) != coin_tosses):
            l_condition.wait()
    
    leader_measurements = measure_leader(player_id)
    # leader_lock.acquire()
    # print(leader_measurements)
    # leader_lock.release()

    coin_measurements = measure_coins(player_id)
    # coin_lock.acquire()
    # print(coin_measurements)
    # coin_lock.release()


In [9]:
threads = []
c_cond = Condition()
l_cond = Condition()
for i in range(num_players):
    t = Thread(target=fail_stop_protocol, args=(i,c_cond,l_cond,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

Player 0 tossed coin for player 0
Player 0 tossed coin for player 1
Player 1 tossed coin for player 0
Player 1 tossed coin for player 1
Player 1 chose leader for player 0
Player 1 chose leader for player 1
Player 0 chose leader for player 0
Player 0 chose leader for player 1
        ┌───┐                ░  ░ ┌─┐               
  q0_0: ┤ H ├──■─────────────░──░─┤M├───────────────
        ├───┤  │             ░  ░ └╥┘┌─┐            
  q0_1: ┤ H ├──┼────■────────░──░──╫─┤M├────────────
        ├───┤  │    │        ░  ░  ║ └╥┘┌─┐         
  q0_2: ┤ H ├──┼────┼────■───░──░──╫──╫─┤M├─────────
        └───┘┌─┴─┐  │    │   ░  ░  ║  ║ └╥┘┌─┐      
  q1_0: ─────┤ X ├──┼────┼───░──░──╫──╫──╫─┤M├──────
             └───┘┌─┴─┐  │   ░  ░  ║  ║  ║ └╥┘┌─┐   
  q1_1: ──────────┤ X ├──┼───░──░──╫──╫──╫──╫─┤M├───
                  └───┘┌─┴─┐ ░  ░  ║  ║  ║  ║ └╥┘┌─┐
  q1_2: ───────────────┤ X ├─░──░──╫──╫──╫──╫──╫─┤M├
                       └───┘ ░  ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/═══════════════════════════